This is simple system to generate specific tests for the students. You need two files: 
* `courseid_22740_participants.csv`: a CSV file containing the names and e-mails of the students (obtained from Moodle): the caolumns should be named `First name`, `Surname` and Èmail address`
* `test.csv`: a CSV file containinf the questions, in the following format:

Points  |  Question   |  data1 

within the`Question` you may use the replacement tag `XXX`, that will be substituted by the info that is under `data1`. It should be straighforward to add a data2 and a second tag, let's say, `YYY`. `data1` should contain a comma separated list of possible substitutions for `XXX`.

Improvements needed:
* to allow for random selection of the `data1` elements, in order to allow for shorter list of elements. Currently you need at least the same number of `data1` options than elements are in the student's list.
* files are created in a given folder, assuming google drive, and links are created ina  file called `links.txt`. It should be possible to generate a list of e-mails for each file without expecting the students to ask for permissions to check their files.


In [1]:
import pandas as pd
from unidecode import unidecode as ud

#
# this is the path for all files and folders. It is set up to deal with 
# Google stream, as the different folders are assigned to each student,
# but of course it can be moved to any other folder
# Note on using Google stream: first time the files with the exams do not exist, which implies
#       that the 'xattr' instruction below does not get the correct ID for the generated files
#       and assign them local IDs instead of google drive's. Run the script twice to ensure the 
#       proper collection of information for the files ID
#
#path=r'/Volumes/GoogleDrive/La meva unitat/Departament/Docència/Biocomputing Area/Interacció Bimolècules (3er)/2020-2021/test/'
path=r'/Volumes/GoogleDrive/La meva unitat/Departament/Docència/Biocomputing Area/Intro Bioinfo (2on)/2020-2021/exams/'
ud(path)
#
# the file containing the students eith their 'First name', 'Surname' and 'Email address'
#students='courseid_22740_participants.csv'
students='courseid_22726_participants.csv'

#
# the file containing the questions, |-separated and with the fields 
# 'Points' | ' Questions' | 'data1' | 'data2' ...
# (rigth now only 'data1' is read, but extending it is straightforward)
#
test='test.csv'

#
# let us first create a couple of data frames with all the information
#
class_df = pd.read_csv(path+students).sort_values(by=['Surname'])
class_df = class_df.reset_index(drop=True)
test_df = pd.read_csv(path+test,sep='|',engine='python')
print(class_df)
print(test_df)

           First name                Surname                 Email address
0               Júlia             ALONSO TOR         juliaalonsotor@uic.es
1               María     ALTARRIBA CODINACH         mariaaltarriba@uic.es
2             Adriana       AMBROSINI GRACIA        adrianambrosini@uic.es
3               Carla          BENET SANCHEZ                cbenets@uic.es
4   Daniela Alejandra      BLANCO ESCAURIAZA         daniela_blanco@uic.es
5               Paula             BOLDO COBO             paulaboldo@uic.es
6               Pablo         BOTELLA MORENO     pablobotellamoreno@uic.es
7                Luis          BOTELLA ROBIN            luisbotella@uic.es
8              Ainhoa            CALVO CALVO            ainhoacalvo@uic.es
9               Laura       CANADELL DEL RIO         lauracanadelld@uic.es
10    Victoria Hannah      CAPRILES HINOJOSA            vickyhannah@uic.es
11              Lluís        CASANOU LABORDA           lluis.caslab@uic.es
12          Guillermo    

In [2]:
import os, random
#
# the file that will contain the links to be sent to the students
#
links=path+'links.txt'
fhl=open(links,'w')

mainInfo = """
How to answer this exam: You have 90 minutes to complete the test. Edit this file only with your answers, and 
do not share it with anyone. Edit this file using your favorite word processor (MSWord, 
Google docs, etc). After finnishing the exam, create a PDF file and submit it to the 
assignment created in the Moodle course. There are 5 file submission attempts allowed. 
When the 90 minutes have expired, the assignment will close and no further submissions 
will be possible. 
""".replace('\n','')
#
# let us iterate over each student, and then over the questions with their 'data1' specificities
# we will create a directory per student within 'path'
#

for ind in class_df.index:  
    personalID = class_df['Email address'][ind].split('@')[0]
    folder = path + '/' + personalID
    file=folder+'/parcial_'+personalID+'.txt'
    #
    # create the folder for each student in case it does not exist
    #
    if not os.path.exists(folder):
        os.makedirs(folder)
    fh=open(file,'w')
    who=class_df['First name'][ind]+' '+class_df['Surname'][ind]
    fh.write(who+'\n'+mainInfo+'\n\n\n')
    #
    # now iterate the exercises in the test
    #
    for ind2 in test_df.index:
        question=test_df['Question'][ind2]
        title='QUESTION '+str(ind2+1)+' ('+str(test_df['Points'][ind2])+' points):'+2*'\n'
        if 'XXX' in question:
            #
            # if the question has some specific tags for substitution, 
            # use the data in 'data1'
            #
            #data=random.choice(str(test_df['data1'][ind2]).split(';'))
            data=str(test_df['data1'][ind2]).split(';')[ind]
            question=question.replace('XXX',data)

            #fh.write(title+question.replace('XXX',data)+3*'\n') 
        if 'YYY' in question:
            data=str(test_df['data2'][ind2]).split(';')[ind]
            question=question.replace('YYY',data)
        if 'ZZZ' in question:
            data=str(test_df['data3'][ind2]).split(';')[ind]
            question=question.replace('ZZZ',data)

            #fh.write(title+question.replace('XXX',data)+3*'\n') 
        fh.write(title+question+3*'\n') 
    fh.close()
    #
    # when using Google drive, it is good to get the info 
    ID = !xattr -p user.drive.id '{folder}'
    infoLink = who+'\thttps://drive.google.com/drive/u/0/folders/'+ID.n
    fhl.write(infoLink+'\n')
    print(infoLink)
fhl.close()


Júlia ALONSO TOR	https://drive.google.com/drive/u/0/folders/155_qEc7Q_kNFLZjHFhiR1QI5cr4lUqwL
María ALTARRIBA CODINACH	https://drive.google.com/drive/u/0/folders/156kmB3ypHpAt3sQhNPx_DvOJYnan3OhA
Adriana AMBROSINI GRACIA	https://drive.google.com/drive/u/0/folders/155avTQNeicTN44DYVyJAZGfZbmceGFRk
Carla BENET SANCHEZ	https://drive.google.com/drive/u/0/folders/159V10ucTceJZoM90u6SxJR0lAK3mIBeZ
Daniela Alejandra BLANCO ESCAURIAZA	https://drive.google.com/drive/u/0/folders/1578zeY3Fw2gxdNLbLuPA12KB1-TIscCQ
Paula BOLDO COBO	https://drive.google.com/drive/u/0/folders/15GURFhcOUox5sXB3uBcZ2sKBIjbKHbqc
Pablo BOTELLA MORENO	https://drive.google.com/drive/u/0/folders/15GSlPYYnzqSctnhGWqM5Enw-Ec7Qh_rM
Luis BOTELLA ROBIN	https://drive.google.com/drive/u/0/folders/15KbNEDc4yh0PpMs8L3aweaNVZUXpxb-2
Ainhoa CALVO CALVO	https://drive.google.com/drive/u/0/folders/15MxVFJLCeEYGG3eeUuydAas-zttvKpUC
Laura CANADELL DEL RIO	https://drive.google.com/drive/u/0/folders/15M5SjKaUcJrbjdHa3Ewqg6xgeg-oEY0o
Victoria